## 📘 Notebook 2: Gộp dữ liệu gốc + lỗi CTGAN và phân tích trực quan

In [ ]:
!pip install -q pandas scikit-learn matplotlib seaborn scipy

In [ ]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import jensenshannon
from scipy.stats import entropy
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from google.colab import files

In [ ]:
# Tải 2 file: dữ liệu gốc và dữ liệu sinh từ CTGAN
uploaded = files.upload()
df_real = pd.read_csv('processed_iot_dataset.csv')
df_fake = pd.read_csv('ctgan_generated_faults_only.csv')

In [ ]:
# Gộp lại thành một file
df_merged = pd.concat([df_real, df_fake], ignore_index=True)
df_merged = df_merged.sort_values(by=['Year', 'Month', 'Day', 'Hour', 'Minute'])
df_merged.to_csv("ctgan_combined_dataset.csv", index=False)
files.download("ctgan_combined_dataset.csv")

In [ ]:
# 1. Biểu đồ phân phối các đặc trưng cảm biến
sensor_cols = ['Temperature', 'Vibration', 'Pressure', 'Voltage', 'Current',
               'FFT_Feature1', 'FFT_Feature2', 'Anomaly_Score']
df_real_fault = df_real[df_real['Fault_Status'] == 1]
for col in sensor_cols:
    plt.figure(figsize=(6, 3))
    sns.kdeplot(df_real_fault[col], label='Thật', fill=True)
    sns.kdeplot(df_fake[col], label='Sinh', fill=True)
    plt.title(f'Phân phối đặc trưng: {col}')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# 2. Phân phối Fault_Type
plt.figure(figsize=(6, 4))
sns.countplot(x=df_real_fault['Fault_Type'], label='Thật')
sns.countplot(x=df_fake['Fault_Type'], color='orange', alpha=0.5, label='Sinh')
plt.legend()
plt.title("Phân bố loại lỗi (Fault_Type)")
plt.show()

In [ ]:
# 3. Ma trận tương quan (dữ liệu thật vs dữ liệu sinh)
plt.figure(figsize=(8,6))
sns.heatmap(df_real_fault[sensor_cols].corr(), annot=False, cmap='coolwarm')
plt.title('Tương quan cảm biến - Dữ liệu thật')
plt.show()

plt.figure(figsize=(8,6))
sns.heatmap(df_fake[sensor_cols].corr(), annot=False, cmap='coolwarm')
plt.title('Tương quan cảm biến - Dữ liệu sinh')
plt.show()

In [ ]:
# 4. PCA trực quan hóa
pca = PCA(n_components=2)
real_pca = pca.fit_transform(df_real_fault[sensor_cols])
fake_pca = pca.transform(df_fake[sensor_cols])
df_pca = pd.DataFrame(np.vstack([real_pca, fake_pca]), columns=['PC1', 'PC2'])
df_pca['Label'] = ['Thật'] * len(real_pca) + ['Sinh'] * len(fake_pca)
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='Label', alpha=0.5)
plt.title('PCA - Thật vs Sinh')
plt.show()

In [ ]:
# 5. t-SNE trực quan hóa
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
df_all = pd.concat([df_real_fault[sensor_cols], df_fake[sensor_cols]], ignore_index=True)
tsne_result = tsne.fit_transform(df_all)
labels = ['Thật'] * len(df_real_fault) + ['Sinh'] * len(df_fake)
df_tsne = pd.DataFrame(tsne_result, columns=['TSNE1', 'TSNE2'])
df_tsne['Label'] = labels
sns.scatterplot(data=df_tsne, x='TSNE1', y='TSNE2', hue='Label', alpha=0.5)
plt.title('t-SNE - Thật vs Sinh')
plt.show()

In [ ]:
# 6. KL và JS Divergence giữa dữ liệu thật vs sinh
def compute_js(p, q):
    p = np.array(p) + 1e-8
    q = np.array(q) + 1e-8
    p /= p.sum()
    q /= q.sum()
    m = (p + q) / 2
    return 0.5 * (entropy(p, m) + entropy(q, m))

for col in sensor_cols:
    real_hist, _ = np.histogram(df_real_fault[col], bins=50, density=True)
    fake_hist, _ = np.histogram(df_fake[col], bins=50, density=True)
    kl = entropy(real_hist, fake_hist)
    js = compute_js(real_hist, fake_hist)
    print(f"{col}: KL Divergence = {kl:.4f}, JS Divergence = {js:.4f}")